In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 1. Data preparation

First lets import the data and generate one set to process all data at once

In [2]:
from google.colab import files
uploaded = files.upload()

Saving gender_submission.csv to gender_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [38]:
train = pd.read_csv("train.csv")
x_test = pd.read_csv("test.csv")
y_test = pd.read_csv("gender_submission.csv")


In [40]:
x_train = train
y_train = train["Survived"]

In [41]:
x_train['set'], x_test['set'] = 'train', 'test'
combined = pd.concat([x_train, x_test])
combined

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,set
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,test
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,test
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,test
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,test


### Missing values
#### First step is to fill missing values and drop columns we will not be using.

In [42]:
combined.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
set               0
dtype: int64

#### We will not be using Embarked and Cabin for this experiment.

In [43]:
combined = combined.drop(['Cabin', 'Embarked'], axis=1)

#### Only one Fare price is missing. fill it with the median of his Pclass:

In [44]:
pclass = combined.loc[combined.Fare.isnull(), 'Pclass'].values[0]
median_fare = combined.loc[combined.Pclass== pclass, 'Fare'].median()
combined.loc[combined.Fare.isnull(), 'Fare'] = median_fare

#### Missing ages
To fill in the missing ages, we can do something more clever then just take the overal median age. The names contain titles of which some are linked to their age. Master is a younger boy (in general). Lets take the median of each age group.

In [45]:
# Select everything before the . as title
combined['Title'] = combined['Name'].str.extract('([A-Za-z]+)\.', expand=True)
combined['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess',
       'Jonkheer', 'Dona'], dtype=object)

In [46]:
title_reduction = {'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 
                   'Master': 'Master', 'Don': 'Mr', 'Rev': 'Rev',
                   'Dr': 'Dr', 'Mme': 'Miss', 'Ms': 'Miss',
                   'Major': 'Mr', 'Lady': 'Mrs', 'Sir': 'Mr',
                   'Mlle': 'Miss', 'Col': 'Mr', 'Capt': 'Mr',
                   'Countess': 'Mrs','Jonkheer': 'Mr',
                   'Dona': 'Mrs'}
combined['Title'] = combined['Title'].map(title_reduction)
combined['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Rev', 'Dr'], dtype=object)

In [47]:
for title, age in combined.groupby('Title')['Age'].median().iteritems():
    print(title, age)
    combined.loc[(combined['Title']==title) & (combined['Age'].isnull()), 'Age'] = age

Dr 49.0
Master 4.0
Miss 22.0
Mr 30.0
Mrs 36.0
Rev 41.5


<ipython-input-47-f6c206bb869f>:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for title, age in combined.groupby('Title')['Age'].median().iteritems():


In [48]:
combined.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
set              0
Title            0
dtype: int64

# 2. Create additional features

Get family size from Parch and Sibsp

In [49]:
combined['family_size'] = combined['Parch'] + combined['SibSp']

KNN needs numerical features therefore, we will convert them to numbers. In a general sense, binary categorical data can work. For larger categorical groups, it only makes sense when the numerical values itself have meaning. For example, for class levels, the difference between first class and third class actually mean something. On the other hand, if we would convert Embarked to a number, there is no meaning in the difference between embarked 1 and embarked 2.

In [50]:
combined['Sex'] = LabelEncoder().fit_transform(combined['Sex'])

First bin the Fare and Age. The groups are choosen arbitrarily:

In [51]:
combined.loc[:, 'Age'] = pd.qcut(combined['Age'], 4, labels=False)
combined.loc[:, 'Fare'] = pd.qcut(combined['Fare'], 5, labels=False)

<ipython-input-51-19aaeb462c18>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  combined.loc[:, 'Age'] = pd.qcut(combined['Age'], 4, labels=False)
<ipython-input-51-19aaeb462c18>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  combined.loc[:, 'Fare'] = pd.qcut(combined['Fare'], 5, labels=False)


#### Select only coluns we will use and scale them

In [52]:
selected = ['Pclass', 'Sex', 'Age', 'Fare', 'family_size']
scaler  = StandardScaler()
scaler.fit(combined[selected])
combined[selected] = scaler.transform(combined[selected])

In [53]:
combined.to_parquet('titanic_family_survivabillity.parquet', index=False)

In [54]:
train = combined.loc[combined['set'] == 'train'].drop('set', axis=1).reset_index(drop=True)
test = combined.loc[combined['set'] == 'test'].drop(['set'], axis=1).reset_index(drop=True)

# 3. Algorithm
KNN works by finding, as the name suggests, the nearest neighbor. It assumes that classes share similar properties. To compare to points, we have to see them as vectors. Each feature adds to a dimension. Would we only have a single feature it would be the numeric distance between the two points. With two or more features, we need to do something smarter. To create a single number to express the similarity (or distance) we need to agegrate all the features (or dimensions). One way is to use the Euclidean distance, which is the root of the sum of squared distances between all features. There are many other ways to agegrate the feature distances, all with their strengths and weaknesses. For this example, we will use the Euclidean distance.

In [55]:
def euclidean_distance(vector1, vector2):
    return np.sqrt(np.sum((vector1 - vector2)**2))

# test function
vec1 = np.array([3, 0])
vec2 = np.array([0, 4])

# this is the 3:4:5 triangle and therefore, it should return 5 (Long live Pythagoras)
euclidean_distance(vec1, vec2)

5.0

Using our distance function we will now find the closest match in our dataset, when providing a vector.

In [56]:
# A first implementation
def get_nearest_neighbor(vector, dataset, number_of_neighbors=1, ignore_cols=['Survived']):
    distances = []
    for ix, row in dataset.loc[:, ~dataset.columns.isin(ignore_cols)].iterrows():
        distance = euclidean_distance(row, vector)
        distances.append((distance, ix))
    indices = [x[1] for x in sorted(distances, key=lambda x: x[0])]
    neighbors = dataset.loc[indices[:number_of_neighbors]]
    return neighbors

# Another implementation using Pandas
def get_nearest_neighbor(vector, dataset, number_of_vectors=1, ignore_cols=['Survived'], not_count_duplicates=False):
    ds = dataset.copy()
    ds['distance'] = ds.loc[:, ~ds.columns.isin(ignore_cols)].apply(
        lambda x: euclidean_distance(x, vector), axis=1)
    if not_count_duplicates:
        distances = sorted(ds.distance.unique())[:number_of_vectors]
        return ds.loc[ds.distance <= max(distances)].drop('distance', axis=1)
    return ds.sort_values('distance', ascending=True).head(number_of_vectors).drop('distance', axis=1)
        
# test function
dataset = pd.DataFrame([
    {'a': 1, 'b': 1, 'Survived': 1},
    {'a': 2, 'b': 2, 'Survived': 1},
    {'a': 3, 'b': 3, 'Survived': 0},
    {'a': 4, 'b': 4, 'Survived': 0},
    {'a': 5, 'b': 5, 'Survived': 0},
])
vector = pd.Series({'a': 2.5, 'b': 2.5})

# should be (2,2) and (3,3) (if keeping track of duplicates)
get_nearest_neighbor(vector, dataset)

,a,b,Survived
1,2,2,1


Now that we have a function to list the nearest neighbors, we need to select the most dominant class of those neighbors to select as our prediction.

In [57]:
def predict(vector, dataset, number_of_neighbors=1, y='Survived'):
    neighbors = get_nearest_neighbor(vector, dataset, number_of_neighbors)
    return round(neighbors[y].mean())

# test function
print(predict(vector, dataset))
print(predict(pd.Series({'a': 4.5, 'b': 4.5}), dataset))

1
0


# 4. Test algorithm on Titanic dataset

To test, we select on row from the set and use KNN to find the best candidate. Of course, we will remove that row from the dataset, or we would find a distance of zero as the identical row is in there.

In [58]:
def predict_dataset(dataset, number_of_neighbors=1):
    ds = dataset.copy()
    def predict_row(vector, dataset):
        subset = dataset.loc[~(dataset.index==vector.name)]
        if vector.name % 100 == 0:
            print(vector.name)
        return int(predict(vector, subset, number_of_neighbors))

    ds['predicted'] = ds.loc[:, ds.columns.isin(selected)].apply(
        lambda x: predict_row(x, ds), axis=1)
    
    return ds

ds = predict_dataset(train, number_of_neighbors=10)

print('Accuracy:', sum(ds['Survived'] == ds['predicted']) / len(ds))


0
100
200
300
400
500
600
700
800
Accuracy: 0.8114478114478114


We find a accuracy of 83.5% on the training set. Lets now make our test set predictions.

## 5. create prediction for testset

In [62]:
def predict_testset(test_dataset, train_dataset, number_of_neighbors=1):
    ds = test_dataset.copy()
    select = selected + ['Survived']
    
    def predict_row(vector, dataset):
        if vector.name % 100 == 0:
            print(vector.name)
        return int(predict(vector, dataset[select], number_of_neighbors))

    ds['Survived'] = ds.loc[:, ds.columns.isin(selected)].apply(
        lambda x: predict_row(x, train_dataset), axis=1)
    
    return ds

In [64]:
final_test = predict_testset(test,train, number_of_neighbors=10)

0
100
200
300
400


In [65]:
result = final_test[['PassengerId', 'Survived']].copy()

In [66]:
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
